## (Упрощенная) ERD сущностей в Clickup
![test](ClickupERD.png "Title")

 Команда - основная верхнеуровневая сущность. Название говорит само за себя. Нас интересует исключительно в разрезе team_id, который используется в некоторых эндпойнтах API
 В команде может быть несколько воркспейсов, в которых, в свою очередь, содержатся листы, которые могут быть сгруппированы в папки, но могут быть и прямыми детьми воркспейса.
 Лист - основная сущность, интересующая нас в контексте сборки датасетов, т.к. это коллекция тасков, которые, в свою очередь, содержат всю интересующую нас информацию (дата создания, статус и т.п.). Стоит обратить внимание, что мы можем определить кастомные поля для использования в тасках (например, причина отвала и т.п.). При запросе к API эти поля будут доступны по отдельному ключу custom_fields.
В интерфейсе все это выглядит как-то так:

![Interface](Entities.png "Ent")

In [198]:
import pandas as pd
import json
import ClickUpAPI as cua
import numpy as np
from ast import literal_eval
from math import ceil
from typing import List
from urllib.parse import quote_plus
import os

Первым делом, как и для практически любого API, нам понадобится токен. В Кликапе есть два вида токенов: персональный, который можно получить в секции Apps настроек своего профиля и полноценный authorization code grant OAuth2. Здесь мы будем использовать персональный токен. ![SegmentLocal](PersToken3.gif "segment")

In [199]:
token = os.environ.get("ClickUpToken")

In [200]:
# Проверим, правильно ли API возвращает юзера, от лица которого делаются запросы:
test_class = cua.ClickupClient(token)
res = test_class.get_user()
print(res)

{'id': 8867829, 'username': 'Bogdan Pilyavets', 'email': 'rupilbo@yandex-team.ru', 'color': '#795548', 'profilePicture': None, 'initials': 'BP', 'week_start_day': None, 'global_font_support': False, 'timezone': 'Europe/Moscow'}


In [201]:
# Получим айдишник команды, к которой у нас есть доступ:
team_id = test_class.get_teams(id_only=True)
print(team_id)

4514698


In [202]:
# получим всю инфу про интересующий нас спейс (в данном случае CRM)
space = test_class.get_space_by_name(team_id, name='CRM')
print(space)

{'id': '6849848', 'name': 'CRM', 'private': False, 'statuses': [{'id': 'p6849848_hGuYhhYV', 'status': 'Open', 'type': 'open', 'orderindex': 0, 'color': '#d3d3d3'}, {'id': 'p6849848_aJ5x29At', 'status': 'concept', 'type': 'custom', 'orderindex': 1, 'color': '#FFCC00'}, {'id': 'p6849848_uHca5I69', 'status': 'in progress', 'type': 'custom', 'orderindex': 2, 'color': '#FF540D'}, {'id': 'p6849848_rQtXfgXn', 'status': 'running', 'type': 'custom', 'orderindex': 3, 'color': '#FF00DF'}, {'id': 'p6849848_Yuo3uSOo', 'status': 'review', 'type': 'custom', 'orderindex': 4, 'color': '#A875FF'}, {'id': 'p6849848_e0PVKVnw', 'status': 'Closed', 'type': 'closed', 'orderindex': 5, 'color': '#6bc950'}], 'multiple_assignees': True, 'features': {'due_dates': {'enabled': True, 'start_date': True, 'remap_due_dates': False, 'remap_closed_due_date': False}, 'sprints': {'enabled': False}, 'points': {'enabled': False}, 'custom_items': {'enabled': False}, 'priorities': {'enabled': True, 'priorities': [{'id': '1', '

In [203]:
# получим всю инфу про интересующие нас листы (в данном случае 'Учащиеся' и 'Лиды (родители)')
studs = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Учащиеся')
pars = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Лиды (родители)')

In [204]:
tags = test_class.get_tags(space['id'])
tags_names = [tag['name'] for tag in tags['tags']]
tags_to_keep = [quote_plus(tag) for tag in tags_names if tag not in ['тестовый', 'технический']]
tags_to_keep.append('')

In [205]:
#заберем данные по таскам 
tasks = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids=[pars['id']])

page=0&list_ids[]=44610695&include_closed=true
page=1&list_ids[]=44610695&include_closed=true
page=2&list_ids[]=44610695&include_closed=true
page=3&list_ids[]=44610695&include_closed=true
page=4&list_ids[]=44610695&include_closed=true
page=5&list_ids[]=44610695&include_closed=true
page=6&list_ids[]=44610695&include_closed=true
page=7&list_ids[]=44610695&include_closed=true
page=8&list_ids[]=44610695&include_closed=true
page=9&list_ids[]=44610695&include_closed=true
page=10&list_ids[]=44610695&include_closed=true
page=11&list_ids[]=44610695&include_closed=true
page=12&list_ids[]=44610695&include_closed=true
page=13&list_ids[]=44610695&include_closed=true
page=14&list_ids[]=44610695&include_closed=true
page=15&list_ids[]=44610695&include_closed=true
page=16&list_ids[]=44610695&include_closed=true
page=17&list_ids[]=44610695&include_closed=true
page=18&list_ids[]=44610695&include_closed=true
page=19&list_ids[]=44610695&include_closed=true
page=20&list_ids[]=44610695&include_closed=true
pa

In [206]:
#поскольку get_all_tasks возвращает нам список nested джейсонов, напишем функцию, которая сложит все это добро в один датафрейм
#подробнее тут https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html
def normalize_wrapper(data:List[dict], **kwargs) -> pd.DataFrame:
    """
    replaces the empty list with a dummy dictionary
    """
    for i in data:
        if not i['tags']:
            i['tags']=[{'name': "fake"}]  #TODO параметризовать ключи
    return pd.json_normalize(data, **kwargs)

def df_getter(data:List[dict], key_getter='tasks', **kwargs) -> pd.DataFrame:
    """
    transforms a list of jsons into a single dataframe using pd.json_normalize
    """
    res = list()
    for chunk in data:
        res.append(normalize_wrapper(chunk[key_getter], **kwargs))
    return pd.concat(res)

In [207]:
par_names = df_getter(tasks,
                      record_path='custom_fields',
                      meta=['id', 'name', 'text_content', 'date_created', 'date_updated', 'date_closed',
                           ['status', 'status'], ['status', 'type'],
                           ['creator', 'email'], 'due_date', 'start_date', 'tags'],
                      meta_prefix='standard_')
#в последней странице почему-то остается только словарь, сейчас нет времени разбираться
missing_list = par_names.loc[par_names['standard_tags']=={'name': 'fake'}]     
par_names.loc[par_names['standard_tags']=={'name': 'fake'}, 'standard_tags'] = pd.Series([[{'name': 'fake'}]]*len(missing_list))

In [208]:
# вытащим теги, чтобы потом выкинуть тестовиков
tags = par_names[['standard_tags', 'standard_id']]
#отберем релевантные столбцы и пивотнем кастомные поля
par_names = par_names.loc[
    par_names['name'].isin([
        'Источник трафика',
        'Канал привлечения',
        'причина отказа',
        'Дата заявки',
        'campaign',
        'content/adset',
        'medium (cpc|cpm)',
        'source (UTM CRM)',
        'причина отказа']), 
        ['name', 'value', 'standard_name', 'standard_id', 'standard_text_content']
        ]
par_names = par_names.pivot(index=['standard_name', 'standard_id', 'standard_text_content'], columns='name', values='value').reset_index()
par_names = par_names.set_index('standard_id')
par_names = par_names.rename(columns={'campaign': 'utm_campaign', 'medium (cpc|cpm)':'utm_medium', 'content/adset': 'utm_content', 'source (UTM CRM)': 'utm_source'})
missing_utms = par_names[par_names['utm_campaign'].isna()]


In [209]:
par_names.head()

name,standard_name,standard_text_content,utm_campaign,utm_content,utm_medium,utm_source,Дата заявки,Источник трафика,причина отказа
standard_id,,,,,,,,,
10hrw71,,Профиль ВК https://vk.com/id\n--данные заявки ...,NaN,NaN,cpc,VK,1630458000000,NaN,6
1nuqc7v,+79026965081,"+79026965081\n19551979zm@gmail.com\n8 класс, н...",NaN,NaN,NaN,NaN,1635037200000,NaN,NaN
13ntc54,"......,",Телефон: +79885556155\nПочта: kil@gmail.com\nU...,yandex-math_test-offer_leadgen,int_child_skill,cpc,facebook,1626224400000,fb_leads,0
12gmh8p,...ان شاء اللہ,Телефон: 89285451214\nemail: gssh2@mail.ru\nUT...,math_fb_leadgen_lookalike_A/B_test,Lookalike_Yes,cpc,facebook,1631581200000,utm_source=fb;utm_medium=cpc;utm_campaign= mat...,0
xb99um,.русик (FB leads via Zapier),Телефон: +79288796149\nПочта: maya_shamsieva@m...,yandex-math_test-offer_leadgen,int_child_blue,cpc,facebook,1623027600000,fb_leads,0


In [210]:
cust_fields = test_class.get_custom_fields(pars['id'])
cust_df = pd.json_normalize(cust_fields, record_path='fields')
churn = pd.DataFrame.from_records(cust_df.loc[cust_df['name']=='причина отказа', 'type_config.options'].values[0])
churn = churn[['name', 'orderindex']].rename(columns={'name':'churn_reason'})

In [211]:
par_names = par_names.reset_index().merge(right=churn, how='left', left_on='причина отказа', right_on='orderindex')
par_names = par_names.set_index('standard_id')

In [212]:
#разберем строку с utm-метками и превратим ее в 3 столбца
missing_utms['Источник трафика'] = missing_utms['Источник трафика'].fillna("")
missing_utms['utms'] = missing_utms['Источник трафика'].str.findall(r"(?<==)([\w\-_\s]*)")
missing_utms['keys'] = missing_utms['Источник трафика'].str.findall(r"([\w\-_]+)(?==)")
#astype здесь используется как способ конвертнуть пустые и непустые листы в False и True соответственно
mask = missing_utms['keys'].astype(bool)
antimask = missing_utms['keys'].astype(bool) == False
utms = missing_utms.loc[mask, ('utms', 'keys')]
no_utms = missing_utms.loc[antimask, ('Источник трафика', 'utms')]
#Конструируем словарь utm-ок для тех у кого они заполнены
utms['dict'] = utms.apply(lambda x: {x['keys'][i]:x['utms'][i] for i in range(len(x['keys']))}, axis=1)
#Конструируем словарь utm-ок по умолчанию, для тех у кого они не прокинуты
no_utms['dict'] = no_utms.apply(lambda x: {"utm_source":x['Источник трафика']}, axis=1)
#Собираем это воедино, разбиваем на столбцы и джойним к родительскому датафрейму
full_utms = pd.concat([utms['dict'], no_utms['dict']])
stand_index = full_utms.index
full_utms = pd.DataFrame(full_utms.tolist())
full_utms = full_utms.set_index(stand_index)
utm_keys = ['utm_source', 'utm_campaign', 'utm_medium', 'utm_content']
full_utms = full_utms[utm_keys]
#par_names = pd.concat([par_names, full_utms], axis=1)

In [213]:
par_names.loc[par_names['utm_campaign'].isna(), utm_keys] = par_names.loc[par_names['utm_campaign'].isna(), utm_keys].combine_first(full_utms)

In [214]:
#оставим только релевантные столбцы и выкинем каретки
par_names = par_names.reset_index().rename({'index': 'standard_id'}, axis=1)
par_names = par_names[['standard_name', 'standard_id', 'utm_source', 'utm_campaign', 'utm_medium', 'utm_content', 'churn_reason', 'Дата заявки']]
par_names['standard_name'] = par_names['standard_name'].replace(r'\n', ' ', regex=True)
#Конвертнем дату заявки в дату
par_names['Дата заявки'] = (par_names['Дата заявки'].apply(pd.to_datetime, origin='unix', unit='ms') + pd.Timedelta("3 hours")).dt.date
#почистим utm от лишних пробелов и приведем к нижнему регистру
par_names[utm_keys] = par_names[utm_keys].apply(lambda x: x.str.strip())
par_names[utm_keys] = par_names[utm_keys].apply(lambda x: x.str.lower())

In [215]:
#сформируем список тестовых айдишников
tags['standard_tags'] = tags['standard_tags'].astype(str)
throwaway=tags.loc[tags['standard_tags'].str.contains('тестовый|технический'), 'standard_id'].unique()

In [216]:
#получим плоский лист всех родительских айдишников
parent_ids = np.setdiff1d(par_names['standard_id'].unique(), throwaway)
task_ids = list(parent_ids)
history = test_class.get_time_in_status(task_ids)

In [218]:
#поскольку get_time_in_status возвращает нам список словарей,
#в роли ключей у которых выступают родительские айдишники
#соберем это в единый словарь и превратим в датафрейм
flattened = dict()
for task_collection in history:
    flattened.update(task_collection)
flat_history = {task_id:flattened[task_id]['status_history'] for task_id in flattened}
flat_df = pd.DataFrame.from_dict(flat_history, orient='index')
#сейчас у нас столбцы со словарями для истории каждого статуса - пометим их префиксом
columns = ['col'+str(i) for i in range(len(flat_df.columns))]
flat_df.columns = columns
flat_df = flat_df.reset_index()
#разберем вложенные словари в каждой группе столбцов через json_normalize
#и дропнем столбцы со словарями
flat_df = flat_df.where(flat_df.notna(), lambda x: [{}])
for col in columns:
    flat_df = flat_df.join(pd.json_normalize(flat_df[col]).add_prefix(col+'_'))
flat_df = flat_df.drop(columns=columns)
flat_df.head()


,index,col0_status,col0_color,col0_type,col0_orderindex,col0_total_time.by_minute,col0_total_time.since,col1_status,col1_color,col1_type,...,col13_status,col13_color,col13_type,col13_total_time.by_minute,col13_total_time.since,col14_status,col14_color,col14_type,col14_total_time.by_minute,col14_total_time.since
0,10hph70,0. новый лид,#d3d3d3,open,0,23,1630489085181,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10hpmc4,0. новый лид,#d3d3d3,open,0,13,1630489576694,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10hpy37,0. новый лид,#d3d3d3,open,0,13,1630491471014,3.слот встречи назначен,#f9d900,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10hqae0,0. новый лид,#d3d3d3,open,0,14,1630494122104,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10hqc1v,0. новый лид,#d3d3d3,open,0,11,1630494431348,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [219]:
#превратим нашу таблицу из широкой в длинную и избавимся от префиксов для столбцов с одинановой сутью
#например, для time_by_minute (время в статусе)
melted_df = flat_df.fillna("").melt(id_vars=['index'])
melted_df['variable'] = melted_df['variable'].str.split("_", n=1).str[1]
#выкинем бесполезные столбцы и сделаем родительские айдишники индексами
melted_df = melted_df[~melted_df['variable'].isin(['orderindex', 'color'])]
melted_df = melted_df.set_index('index')
#соберем список отдельных датафреймов для каждого статуса и схлопнем их в одну таблицу
#NB при итерировании по групбаю он возвращает сначала ключ, а потом датафрейм по этому ключу
to_concat = [y for x,y in melted_df.groupby('variable', as_index=False)]
new_df = pd.concat(to_concat, axis=1)
#дропнем нерелевантные столбцы и переименуем оставшиеся
col_names = list(new_df.iloc[0, [i for i in range(0,len(new_df.columns),2)]])
new_df = new_df.iloc[:,[i for i in range(1,len(new_df.columns),2)]]
new_df.columns = col_names
new_df = new_df.rename(columns={'total_time.by_minute':'min_passed', 'total_time.since': 'start_dt'})
#конвертнем столбец со временем в формат даты
new_df['start_dt'] = new_df['start_dt'].apply(pd.to_datetime, origin='unix', unit='ms') + pd.Timedelta("3 hours")
new_df.head()

,status,min_passed,start_dt,type
index,,,,
10hph70,0. новый лид,23,2021-09-01 12:38:05.181,open
10hpmc4,0. новый лид,13,2021-09-01 12:46:16.694,open
10hpy37,0. новый лид,13,2021-09-01 13:17:51.014,open
10hqae0,0. новый лид,14,2021-09-01 14:02:02.104,open
10hqc1v,0. новый лид,11,2021-09-01 14:07:11.348,open


In [220]:
#получим список всех детей из кликапа
kids = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids=[studs['id']])

page=0&list_ids[]=46598432&include_closed=true
page=1&list_ids[]=46598432&include_closed=true
page=2&list_ids[]=46598432&include_closed=true
page=3&list_ids[]=46598432&include_closed=true
page=4&list_ids[]=46598432&include_closed=true
page=5&list_ids[]=46598432&include_closed=true


In [221]:
#как и родителями, сложим все в один датафрейм
df = df_getter(kids,
               record_path='custom_fields',
               meta=['id', 'name', 'text_content', 'date_created', 'date_updated', 'date_closed',
                    ['status', 'status'], ['status', 'type'],
                    ['creator', 'email'], 'due_date', 'start_date'],
               meta_prefix="standard_")
df.columns

Index(['id', 'name', 'type', 'date_created', 'hide_from_guests', 'required',
       'type_config.fields', 'type_config.field_inverted_name',
       'type_config.linked_subcategory_access',
       'type_config.subcategory_inverted_name', 'type_config.subcategory_id',
       'value', 'type_config.options', 'standard_id', 'standard_name',
       'standard_text_content', 'standard_date_created',
       'standard_date_updated', 'standard_date_closed',
       'standard_status.status', 'standard_status.type',
       'standard_creator.email', 'standard_due_date', 'standard_start_date'],
      dtype='object')

In [222]:
#выкинем тестовых юзеров
df = df[df['standard_status.status']!='тестовый']
df.sample(5)

,id,name,type,date_created,hide_from_guests,required,type_config.fields,type_config.field_inverted_name,type_config.linked_subcategory_access,type_config.subcategory_inverted_name,...,standard_name,standard_text_content,standard_date_created,standard_date_updated,standard_date_closed,standard_status.status,standard_status.type,standard_creator.email,standard_due_date,standard_start_date
477,a8df8f2b-0eab-4b11-aedc-26f18397c902,комментарий для преподавателя,text,1612451352639,False,False,NaN,NaN,NaN,NaN,...,Трофим Лодойбалов,\n,1627217290435,1633608772946,None,не планирует покупку,custom,s-dayana21@yandex-team.ru,1633568400000,None
852,a8df8f2b-0eab-4b11-aedc-26f18397c902,комментарий для преподавателя,text,1612451352639,False,False,NaN,NaN,NaN,NaN,...,Матвей Якунин,\n,1618246021771,1634659921289,None,не планирует покупку,custom,rahmatulina@yandex-team.ru,1634605200000,None
913,21542971-21af-4389-aaae-e5a56e4c5fde,купленные пакеты,list_relationship,1615539828118,False,False,[{'field': 'cf_aef0fa70-5268-43e6-9fab-5b91a05...,Ребенок в CRM,True,Заявки на покупку,...,Саша Яковлев,\n,1626425489932,1633524512373,None,планирует покупку,custom,s-dayana21@yandex-team.ru,1636101000000,None
408,57b23809-f60d-4b04-9978-2d48574f2c3e,Пройденные занятия пакета (старое),short_text,1613133715596,False,False,NaN,NaN,NaN,NaN,...,Диана Штанникова,\n,1627317410097,1634039135283,None,не планирует покупку,custom,k-perfileva@yandex-team.ru,1634039100000,None
1115,a7d9f126-7ed3-48ea-8a1d-adcf5b1733fb,Родитель в воронке лидов,list_relationship,1613761441536,False,False,"[{'name': None, 'field': 'cf_a8df8f2b-0eab-4b1...",Учащийся,True,Лиды (родители),...,Арина Савельева,\n,1629211114412,1630059429191,None,возврат,custom,s-dayana21@yandex-team.ru,None,None


In [223]:
#удалим не нужные нам столбцы
to_drop = ['type_config.default', 'type_config.placeholder', 'type_config.new_drop_down', 'type_config.options',
       'type_config.fields', 'type_config.field_inverted_name',
       'type_config.linked_subcategory_access',
       'type_config.subcategory_inverted_name', 'type_config.subcategory_id']
df = df.drop(columns=df.columns.intersection(to_drop))
df.head()

,id,name,type,date_created,hide_from_guests,required,value,standard_id,standard_name,standard_text_content,standard_date_created,standard_date_updated,standard_date_closed,standard_status.status,standard_status.type,standard_creator.email,standard_due_date,standard_start_date
0,9fb75587-2201-4dc0-9e3d-78f6f2e0c69a,Заявка на допродажу,list_relationship,1633346052564,False,False,NaN,1p0dpev,Полина Кибардина,,1635177280185,1635177315192,None,первая покупка,custom,bogushsemen@yandex-team.ru,1635246900000,None
1,ec8cfddf-c57c-4971-bacf-fbb61975f3fd,Пары,list_relationship,1633348018326,False,False,NaN,1p0dpev,Полина Кибардина,,1635177280185,1635177315192,None,первая покупка,custom,bogushsemen@yandex-team.ru,1635246900000,None
2,8c9dfbda-6c96-41e4-a746-6df5b51e2727,Преподаватель списка учащиеся,short_text,1612453867921,False,False,NaN,1p0dpev,Полина Кибардина,,1635177280185,1635177315192,None,первая покупка,custom,bogushsemen@yandex-team.ru,1635246900000,None
3,57b23809-f60d-4b04-9978-2d48574f2c3e,Пройденные занятия пакета (старое),short_text,1613133715596,False,False,NaN,1p0dpev,Полина Кибардина,,1635177280185,1635177315192,None,первая покупка,custom,bogushsemen@yandex-team.ru,1635246900000,None
4,4a4363cf-e27e-4b59-b781-63cb161a43e3,Расписание,short_text,1613142022298,False,False,NaN,1p0dpev,Полина Кибардина,,1635177280185,1635177315192,None,первая покупка,custom,bogushsemen@yandex-team.ru,1635246900000,None


In [224]:
#пивотнем столбец name (каждое уникальное значение из этого столбца становится отдельным столбцом в пивотизированном датафрейме)
#поскольку столбец 'родитель в воронке лидов' в кликапе является ссылкой на другой лист, API возвращает нам этот лист, а пандас
#читает его как питоновский лист. В нашем случае в нем всегда один элемент, поэтому explode не увеличивает кол-во строк в датафрейме
#а просто избавляет нас от листа в cтолбце, превращая ее в столбец словарей
df2 = df.copy()
df2 = df2.pivot(index='standard_id', columns='name', values='value')
df2 = df2.explode('Родитель в воронке лидов')
df2.head()

name,Shared with me,Заявка на допродажу,Инциденты,Пары,Преподаватель списка учащиеся,Прогулы,Пройденные занятия пакета (старое),Расписание,Родитель в воронке лидов,Согласие на видеозапись,Цель ученика и сроки достижения цели,класс,комментарий для преподавателя,купленные пакеты,первое занятие
standard_id,,,,,,,,,,,,,,,
10htvj3,NaN,"[{'id': '1g5gmfa', 'name': 'Борис Севастьянов ...",NaN,NaN,NaN,"[{'id': '118uqak', 'name': 'Борис Севастьянов'...",NaN,NaN,"{'id': 'z9w1ab', 'name': 'Ксения Екимова', 'st...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,NaN,"[{'id': '10htu81', 'name': 'Борис Севастьянов'...",NaN
118tu1c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'id': 'wkyr6z', 'name': 'Елена Маклига', 'sta...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,NaN,"[{'id': '118tt2r', 'name': 'Елена Маклига', 's...",NaN
118tz1t,NaN,"[{'id': '1k4v2p4', 'name': 'Александр Крок', '...",NaN,NaN,Екатерина Пинаева,"[{'id': '1fu5xur', 'name': 'Анна Крок', 'statu...",NaN,"вт 20:00, пт 19:00","{'id': '118pbfa', 'name': 'Анастасия Крок', 's...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,7,NaN,"[{'id': '118ty1k', 'name': 'Александр Крок', '...",NaN
118u1dz,NaN,NaN,NaN,NaN,Георгий Поседко,NaN,NaN,ВС. 12:00 – 13:00,"{'id': 'z9watm', 'name': 'Екатерина Александро...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,NaN,"[{'id': '118u1a2', 'name': 'Екатерина Александ...",NaN
118uqam,NaN,NaN,NaN,NaN,Георгий Поседко,"[{'id': '1g006x5', 'name': 'Андрей Преображенс...",NaN,ЧТ. 18:00 – 19:00 СБ. 15:00 – 16:00,"{'id': 'yyd589', 'name': 'Ирина Преображенская...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,7,NaN,"[{'id': '118up13', 'name': 'Андрей Преображенс...",NaN


In [225]:
#читаем наш столбец со словарями в отдельный датафрейм, добавляем префикс к именам столбцов
#чтобы избежать конфликтов при последующем мердже
parent = df2['Родитель в воронке лидов'].apply(pd.Series).add_prefix("par_")
parent = parent[['par_id', 'par_name']]
parent.head()

,par_id,par_name
standard_id,,
10htvj3,z9w1ab,Ксения Екимова
118tu1c,wkyr6z,Елена Маклига
118tz1t,118pbfa,Анастасия Крок
118u1dz,z9watm,Екатерина Александрова
118uqam,yyd589,Ирина Преображенская


In [226]:
#проделываем то же самое для столбца со ссылкой на купленные пакеты
packages = df2['купленные пакеты'].explode().apply(pd.Series).add_prefix("purch_")
packages = packages[['purch_id']]
packages.head()

,purch_id
standard_id,
10htvj3,10htu81
118tu1c,118tt2r
118tz1t,118ty1k
118u1dz,118u1a2
118uqam,118up13


In [227]:
#получаем инфу о списке покупок в кликапе
purch_list = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Заявки на покупку')
print(purch_list)

{'id': '48853634', 'name': 'Заявки на покупку', 'orderindex': 0, 'status': None, 'priority': None, 'assignee': None, 'task_count': 776, 'due_date': None, 'start_date': None, 'folder': {'id': '23627065', 'name': 'hidden', 'hidden': True, 'access': True}, 'space': {'id': '6849848', 'name': 'CRM', 'access': True}, 'archived': False, 'override_statuses': True, 'permission_level': 'create'}


In [228]:
#забираем все таски из этого списка
purchases = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids = [purch_list['id']])

page=0&list_ids[]=48853634&include_closed=true
page=1&list_ids[]=48853634&include_closed=true
page=2&list_ids[]=48853634&include_closed=true
page=3&list_ids[]=48853634&include_closed=true
page=4&list_ids[]=48853634&include_closed=true
page=5&list_ids[]=48853634&include_closed=true
page=6&list_ids[]=48853634&include_closed=true
page=7&list_ids[]=48853634&include_closed=true
page=8&list_ids[]=48853634&include_closed=true


In [229]:
#и внвь соберем эту инфу в плоский датафрейм, выкинем тесовых и оставим только релевантные столбцы
purch_df = df_getter(purchases, record_path='custom_fields', meta=['id', ['status', 'status']],meta_prefix='standard_')
purch_df = purch_df[(purch_df['name']=='Ссылка на ребенка в админке') & (purch_df['standard_status.status']!='тестовый')][['value', 'standard_id']].dropna()
#вытащим praktikum_id из юрла
purch_df['praktikum_id'] = purch_df['value'].str.split("/").str[-1]
purch_df.head()

,value,standard_id,praktikum_id
3,https://praktikum-admin.yandex-team.ru/math/st...,1p0dmdd,13045672
13,https://praktikum-admin.yandex-team.ru/math/st...,1p0dg51,13045433
23,https://praktikum-admin.yandex-team.ru/math/st...,1p0d23y,13013522
33,https://praktikum-admin.yandex-team.ru/math/st...,1p0ctgw,12486415
43,https://praktikum-admin.yandex-team.ru/math/st...,1p0c54a,13043871


In [230]:
#сопоставим айдишники детей с их айдишниками в практикуме
kids_links = packages.reset_index().merge(purch_df, how='left', left_on='purch_id', right_on='standard_id')
kids_links = kids_links[['purch_id', 'praktikum_id', 'standard_id_x']].groupby('standard_id_x')['praktikum_id'].last()
kids_links.head()

standard_id_x
10htvj3    12363984
118tu1c    12391251
118tz1t    12391974
118u1dz    12392531
118uqam    12394280
Name: praktikum_id, dtype: object

In [231]:
#теперь сджойним детей с их родителями
full_lineage = parent.merge(kids_links, how='left', left_index=True, right_index=True)
full_lineage.sample(5)

,par_id,par_name,praktikum_id
standard_id,,,
p7gq23,15xwwme,Наталья Губарева,11346440
6fynn4,69yhrv,Татьяна Рушечникова (Болгова),7210563
1kq4jnt,12pmkg7,Анна Македонская,12608526
1kq5dhk,12gtg83,Алексей Кобевко,12611889
1jzb99b,1befq6r,Алёна Завьялова,12639738


In [232]:
#наконец, смерджим все это воедино с датафреймом родительских атрибутов
fin_df = new_df.reset_index().merge(full_lineage, how='left', left_on='index', right_on='par_id')
fin_df = fin_df[fin_df['type']!=""]
fin_df = fin_df.merge(par_names, how='left', left_on='index', right_on='standard_id')
fin_df.sample(10)

,index,status,min_passed,start_dt,type,par_id,par_name,praktikum_id,standard_name,standard_id,utm_source,utm_campaign,utm_medium,utm_content,churn_reason,Дата заявки
15653,118m9ke,9. есть оплата и запись,72150.0,2021-09-05 16:27:39.158,custom,118m9ke,Попова Оксана,12385374,Попова Оксана,118m9ke,facebook,math_fb_leadgen_kids&teachers,cpc,fb_inattention,NaN,2021-09-03
2169,1kg8gar,0. новый лид,68,2021-10-02 15:12:52.165,open,NaN,NaN,NaN,Елена Касапова,1kg8gar,facebook,math_fb_leadgen_platform,cpc,platform_women,уже неакт. - отстаньте,2021-10-02
18087,8byk30,"2.перезвонить, узнать слот",4663.0,2021-05-15 11:03:52.599,custom,NaN,NaN,NaN,Екатерина Карева,8byk30,other,NaN,NaN,NaN,(old) уже неактуально,2021-05-10
17397,1m1hzkw,3.слот встречи назначен,1034.0,2021-10-08 20:00:39.991,custom,NaN,NaN,NaN,Галина Шаповалова,1m1hzkw,facebook,math_fb_leadgen_platform_lal_2.0,cpc,platform_lal.2.0,уже неакт. - передумали,2021-10-08
7031,1ahy7fa,1. недозвон 1 сутки,2873.0,2021-08-01 11:47:59.778,custom,NaN,NaN,NaN,Наталья Константинова,1ahy7fa,facebook,yandex-math_test-offer_leadgen,cpc,leto_int_child,NaN,2021-08-01
11642,12pc24q,2.перезвонить на неделе,15645.0,2021-09-16 12:42:24.885,custom,NaN,NaN,NaN,Анастасия,12pc24q,facebook,math_fb_leadgen_kids&teachers - men,cpc,fb_inattention_men,не ЦА - случайно,2021-09-15
11059,11eece2,3.слот встречи назначен,36576.0,2021-09-29 19:42:00.940,custom,11eece2,Артем Субботин,12704573,Артем Субботин,11eece2,,,,,NaN,2021-09-06
152,118t2av,0. новый лид,15,2021-09-03 19:44:17.510,open,NaN,NaN,NaN,Татьяна Кузнецова,118t2av,facebook,math_fb_leadgen_kids&teachers,cpc,fb_teacher,не ЦА - класс,2021-09-03
1703,1bepnk0,0. новый лид,1,2021-08-07 15:56:08.678,open,NaN,NaN,NaN,КЕРАТИН | БОТОКС |КРАСНОДАР|,1bepnk0,facebook,yandex-math_test-offer_leadgen,cpc,int_child_leto_yandex,недозвон,2021-08-07
25460,q1ameh,6.нужна перезапись,3618.0,2021-10-12 17:06:47.994,custom,NaN,NaN,NaN,Галина Миронова,q1ameh,facebook,math_fb_leadgen_september_offer,cpc,offer4_zanimaysya_v_september,недозвон,2021-08-01


In [233]:
#добавим мэппинг статусов из файла
statuses = pd.read_excel('mappings.xlsx', engine='openpyxl', sheet_name='statuses', index_col=1)
fin_df = fin_df.merge(statuses, left_on='status', right_index=True, how='left')
fin_df.head(10)

,index,status,min_passed,start_dt,type,par_id,par_name,praktikum_id,standard_name,standard_id,utm_source,utm_campaign,utm_medium,utm_content,churn_reason,Дата заявки,этап,Созвон,Бронирование,Встреча
0,10hph70,0. новый лид,23,2021-09-01 12:38:05.181,open,NaN,NaN,NaN,Кати Долгова,10hph70,facebook,math_fb_leadgen_lookalike_a/b_test,cpc,lookalike_no,недозвон,2021-09-01,0 заявка,0.0,0.0,0.0
1,10hpmc4,0. новый лид,13,2021-09-01 12:46:16.694,open,NaN,NaN,NaN,Татьяна Гулько,10hpmc4,facebook,math_fb_leadgen_kids&teachers,cpc,fb_study_begin,недозвон,2021-09-01,0 заявка,0.0,0.0,0.0
2,10hpy37,0. новый лид,13,2021-09-01 13:17:51.014,open,10hpy37,Оксана Маришкина,12484377,Оксана Маришкина,10hpy37,yandex,math_yandex_search,cpc,NaN,NaN,2021-09-01,0 заявка,0.0,0.0,0.0
3,10hqae0,0. новый лид,14,2021-09-01 14:02:02.104,open,10hqae0,Наталья Цибульник,12635010,Наталья Цибульник,10hqae0,facebook,math_fb_leadgen_lookalike_a/b_test,cpc,lookalike_yes,NaN,2021-09-01,0 заявка,0.0,0.0,0.0
4,10hqc1v,0. новый лид,11,2021-09-01 14:07:11.348,open,NaN,NaN,NaN,Нигара Аркин,10hqc1v,facebook,math_fb_leadgen_kids&teachers - lal,cpc,fb_study_begin,недозвон,2021-09-01,0 заявка,0.0,0.0,0.0
5,10hqkbr,0. новый лид,15,2021-09-01 14:37:45.110,open,NaN,NaN,NaN,Елена Лачугина,10hqkbr,facebook,math_fb_leadgen_kids&teachers - lal,cpc,fb_inattention,уже неакт. - отстаньте,2021-09-01,0 заявка,0.0,0.0,0.0
6,10hqrdm,0. новый лид,1,2021-09-01 14:58:44.035,open,NaN,NaN,NaN,Елена,10hqrdm,facebook,math_fb_leadgen_kids&teachers - lal,cpc,fb_inattention,недозвон,2021-09-01,0 заявка,0.0,0.0,0.0
7,10hqwmp,0. новый лид,3,2021-09-01 15:08:59.467,open,NaN,NaN,NaN,Валентина Кузьмич,10hqwmp,facebook,math_fb_leadgen_august,cpc,math_august,уже неакт. - передумали,2021-09-01,0 заявка,0.0,0.0,0.0
8,10hr1ad,0. новый лид,9,2021-09-24 16:12:12.633,open,10hr1ad,Ленара Дегтярева,12731723,Ленара Дегтярева,10hr1ad,facebook,math_fb_leadgen_kids&teachers,cpc,fb_teacher,NaN,2021-09-01,0 заявка,0.0,0.0,0.0
9,10hrnnr,0. новый лид,103,2021-09-01 16:32:24.559,open,10hrnnr,Елена Редько,12449405,Елена Редько,10hrnnr,,NaN,NaN,NaN,NaN,2021-09-01,0 заявка,0.0,0.0,0.0


In [234]:
#создадим отдельные столбцы с датой созвона, брони и встречи
fin_df['call'] = fin_df['start_dt'].where(fin_df['Созвон']==1, np.NaN).groupby(fin_df['index']).transform('min')
fin_df['meeting'] = fin_df['start_dt'].where(fin_df['Встреча']==1, np.NaN).groupby(fin_df['index']).transform('min')
fin_df['booking'] = fin_df['start_dt'].where(fin_df['Бронирование']==1, np.NaN).groupby(fin_df['index']).transform('min')
#выкинем лишние столбцы
fin_df = fin_df.drop(columns=['Созвон', 'Встреча', 'par_name', 'index', 'par_id']).rename(columns={'причина отказа': 'churn_reason'})
fin_df['lead_date'] = fin_df.groupby('standard_id')['start_dt'].transform('min')
fin_df['lead_date'] = fin_df['Дата заявки'].combine_first(fin_df['lead_date'])
#конвертнем дату заявки в datetime, предварительно проверив на пустоту
if len(fin_df.loc[fin_df['Дата заявки'].isna(), 'lead_date']) > 0:
    fin_df.loc[fin_df['Дата заявки'].isna(), 'lead_date'] = fin_df[fin_df['Дата заявки'].isna()]['lead_date'].apply(pd.to_datetime).dt.date
fin_df = fin_df.drop(columns=['Дата заявки', 'Бронирование'])
fin_df = fin_df.rename(columns={'этап': 'etap'})
fin_df.sample(7)

,status,min_passed,start_dt,type,praktikum_id,standard_name,standard_id,utm_source,utm_campaign,utm_medium,utm_content,churn_reason,etap,call,meeting,booking,lead_date
491,0. новый лид,3,2021-09-09 15:20:15.363,open,NaN,Людмила С,11txnen,facebook,math_fb_leadgen_kids&teachers,cpc,fb_study_begin,недозвон,0 заявка,NaT,NaT,NaT,2021-09-09
23294,6. отказ после брони,3101.0,2021-10-23 15:20:21.385,custom,NaN,Ядвига Черепанова,1n9m9v9,facebook,,cpc,,уже неакт. - купили,4 подтверждаем встречу,2021-10-20 13:17:02.380,NaT,2021-10-20 13:17:02.380,2021-10-20
17355,5.встреча подтверждена,1420.0,2021-10-08 09:44:05.657,custom,NaN,Светлана Останина,1m1d4kn,facebook,math_fb_leadgen_platform_lal_2.0,cpc,platform_lal.2.0,не ЦА - прочее,4 подтверждаем встречу,2021-10-08 09:29:27.342,NaT,2021-10-08 09:29:27.342,2021-10-07
15684,1. недозвон (пингуем нед.),50.0,2021-09-12 12:05:20.526,custom,NaN,Юлия,118tb4h,yandex,math_yandex_search,cpc,NaN,уже неакт. - передумали,1 устанавливаем контакт,2021-09-12 12:55:26.241,NaT,NaT,2021-09-03
13649,5.ссылка отправлена,560.0,2021-10-22 09:22:00.286,custom,13001030,Раиса Мингазова,1n9jy1z,facebook,,cpc,,NaN,4 подтверждаем встречу,2021-10-19 19:48:56.613,2021-10-22 18:42:09.061,2021-10-19 19:48:56.613,2021-10-19
11683,2. отказ после контакта,45123.0,2021-09-24 10:56:01.460,custom,NaN,Юлия,12petgf,vk,math_vk_september,cpc,NaN,недозвон,2 выясняем цели,2021-09-16 11:32:37.397,NaT,NaT,2021-09-16
3071,0. новый лид,3,2021-10-17 13:41:03.876,open,NaN,Айна,1my5ym5,facebook,,cpc,,недозвон,0 заявка,NaT,NaT,NaT,2021-10-17


In [235]:
fin_df['standard_id'].nunique()

5385

In [236]:
fin_df.to_csv('fin1_df.csv')

# ЗЭ ЭНД